In [14]:
import polars as pl
import pandas as pd
import json
from pprint import pprint

# import
from langchain_community.document_loaders import DirectoryLoader, JSONLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI, AzureOpenAI
from langchain_openai import ChatOpenAI

In [21]:
config = json.load(open('../config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint']  

In [3]:
label_dat = pl.read_csv('../data/labels_for_rag.csv')
label_dat.head()

SETID,SPL_VERSION,ingredients_names,ingredients_rxcuis,condition_name,pt_meddra_id,pt_meddra_term,file,title
str,i64,str,i64,str,i64,str,str,str
"""0002ad27-779d-…",22,"""trametinib""",1425099,"""GIB""",10052534,"""large intestin…","""../data/2024_l…","""These highligh…"
"""00aa81c2-73c7-…",4,"""esomeprazole""",283742,"""GIB""",10013839,"""duodenal ulcer…","""../data/2024_l…","""These highligh…"
"""00fd1905-27e4-…",11,"""nitisinone""",61805,"""GIB""",10027141,"""melaena""","""../data/2024_l…","""These highligh…"
"""03880372-2c68-…",21,"""rilpivirine""",1102270,"""ALI""",10019851,"""hepatotoxicity…","""../data/2024_l…","""These highligh…"
"""09beda19-56d6-…",33,"""linaclotide""",1307404,"""GIB""",10038063,"""rectal haemorr…","""../data/2024_l…","""These highligh…"


In [17]:
loader = JSONLoader(
    file_path='../data/2024_latest_labels/latest_labels_ingredients/20240111_6370e342-16a5-4339-a13f-ba275fd5de09.json',
    jq_schema='.sections',
    text_content=False)

data = loader.load()
pprint(data)

[Document(page_content='{"AR": "ADVERSE REACTIONS\\n\\n\\nClinical Trial Adverse Events. Clomiphene citrate, at recommended dosages, is generally well tolerated. Adverse reactions usually have been mild and transient and most have disappeared promptly after treatment has been discontinued. Adverse experiences reported in patients treated with clomiphene citrate during clinical studies are shown in Table 2.\\n\\nTable 2. Incidence of Adverse Events in Clinical Studies (Events Greater than 1%) (n = 8029*)\\n\\n\\n\\n\\n\\n\\n\\u00a0*Includes 498 patients whose reports may have been duplicated in the event totals and could not be distinguished as such. Also, excludes 47 patients who did not report symptom data.\\n\\n\\n\\n\\n\\nAdverse Event\\n\\n\\n%\\n\\n\\n\\n\\nOvarian Enlargement\\n\\n\\n13.6\\n\\n\\n\\n\\nVasomotor Flushes\\n\\n\\n10.4\\n\\n\\n\\n\\nAbdominal-Pelvic Discomfort/Distention/Bloating\\n\\n\\n5.5\\n\\n\\n\\n\\nNausea and Vomiting\\n\\n\\n2.2\\n\\n\\n\\n\\nBreast Discomfo

In [22]:
documents = []
for file in label_dat['file'].unique():
    loader = JSONLoader(file_path='../data/2024_latest_labels/latest_labels_ingredients/20240111_6370e342-16a5-4339-a13f-ba275fd5de09.json',
                        jq_schema='.sections',
                        text_content=False)
    documents.extend(loader.load())

text_splitter = CharacterTextSplitter()
chunks = text_splitter.split_documents(documents)
db = Chroma.from_documents(chunks, OpenAIEmbeddings(openai_api_key=api_key), persist_directory="../../Chroma")
db.persist()

/Users/undinagisladottir/opt/anaconda3/envs/llm_cpus/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
